In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip uninstall transformers torch accelerate -y
!pip install transformers[torch] accelerate

Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: torch 2.3.1
Uninstalling torch-2.3.1:
  Successfully uninstalled torch-2.3.1
Found existing installation: accelerate 0.31.0
Uninstalling accelerate-0.31.0:
  Successfully uninstalled accelerate-0.31.0
  Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.


In [3]:
!pip install seqeval

In [4]:
! pip install evaluate

In [5]:
!pip install datasets

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from datasets import load_dataset,Audio

dataset = load_dataset("csv",data_files="/content/drive/MyDrive/datos_finale.csv",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['label', 'audio'],
    num_rows: 2116
})

In [8]:
dataset = dataset.remove_columns(["ID_frase", "conv","speaker","video_name","emotion_x","text"])
dataset=dataset.rename_column('emocion_code','label')

In [9]:
from datasets import load_dataset, DatasetDict
import numpy as np
import ast
# Definir una función de procesamiento para convertir las cadenas en arrays de numpy
def convert_to_numpy(example):
    example['audio'] = np.array(ast.literal_eval(example['audio']))
    return example

# Aplicar la función de procesamiento al dataset



In [10]:
processed_dataset = dataset.map(convert_to_numpy)

Map:   0%|          | 0/2116 [00:00<?, ? examples/s]

In [11]:
train_testvalid = processed_dataset.train_test_split(test_size=0.2)
test = train_testvalid['test']

train_valid = train_testvalid['train'].train_test_split(test_size=0.25)
train = train_valid['train']
val = train_valid['test']

In [12]:
from transformers import  Wav2Vec2FeatureExtractor, AutoConfig

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

In [13]:
def preprocess_function(examples):
    audio_arrays = examples["audio"]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [14]:
encoded_minds_test = test.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds_train = train.map(preprocess_function, remove_columns="audio", batched=True)
encoded_minds_val = val.map(preprocess_function, remove_columns="audio", batched=True)


Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Map:   0%|          | 0/1269 [00:00<?, ? examples/s]

Map:   0%|          | 0/423 [00:00<?, ? examples/s]

In [15]:
encoded_minds_test

Dataset({
    features: ['label', 'input_values', 'attention_mask'],
    num_rows: 424
})

In [16]:
id2label = {0: "ANGER", 1: "DISGUST",2:"FEAR",3:"JOY",4:"SADNESS",5:"SURPRISE"}
label2id = {"ANGER": 0, "DISGUST": 1,"FEAR":2,"JOY":3,"SADNESS":4,"SURPRISE":5}

In [17]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = 6
model = AutoModelForAudioClassification.from_pretrained(
    "audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim", num_labels=num_labels,id2label=id2label, label2id=label2id
)

config.json:   0%|          | 0.00/2.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/661M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
import evaluate

accuracy = evaluate.load("accuracy")

In [18]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [21]:
training_args = TrainingArguments(
    output_dir="audio_wav2vec_largeRobust",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds_train,
    eval_dataset=encoded_minds_test,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.786800,1.783860,0.287736
2,1.775700,1.764113,0.330189
3,1.751100,1.731276,0.341981
4,1.716700,1.690491,0.327830
5,1.679600,1.642148,0.325472
6,1.645100,1.603839,0.325472
7,1.616300,1.587791,0.325472
8,1.602300,1.582392,0.325472
9,1.598800,1.581015,0.325472
10,1.593500,1.580593,0.325472


KeyboardInterrupt: 

In [ ]:
task_evaluator = evaluator("audio-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline="Drigoro/audio_wav2vec_largeRobust",
    data=encoded_minds_val,
    metric=evaluate.combine(["accuracy"]),
    label_mapping={"LABEL_0": 0, "LABEL_1": 1,"LABEL_2":2,"LABEL_3":3,"LABEL_4":4,"LABEL_5":5}
)
print(eval_results)